In [1]:
import pandas as pd
import numpy as np
import random
import copy
import csv
from collections import Counter

In [2]:
### AUXILIARY FUNCTIONS ###
def NumberOfRounds(number_players):
    if number_players > 256:
        return 8
    elif number_players > 128:
        return 7
    elif number_players > 64:
        return 6
    elif number_players > 32:
        return 5
    elif number_players > 16:
        return 4
    elif number_players > 8:
        return 3
    else:
        print("NUMBER OF ROUNDS?")
        return 0

In [3]:
### GENERAL VARIABLES ###

number_players = 2000 # must be even (byes are ignored)
number_tables = int(number_players / 2)
total_rounds = NumberOfRounds(number_players)
id_list = [n for n in range(number_players)]

In [4]:
### DEFINING CLASSES ###
class Deck:
    def __init__(self, name):
        self.name = name
        self.winrates = []
        self.tierates = []
        
    def generateMatchupData(self, total_data_winrate, total_data_tierate):
        # create dict with {"Deck": [44%, 2%]} # first value is winrate, second is tie rate
        self.winrates = total_data_winrate[self.name]
        self.tierates = total_data_tierate[self.name]

class Player:
    def __init__(self, id, deck):
        self.id = id
        self.deck = deck
        self.roundData = []
        self.previousOpps = []
        self.score = 0
        
    def createRound(self, opponent, result):
        self.roundData.append([opponent, result])
        self.previousOpps.append(opponent)
        self.score += result
        
    def matchHistory(self):
        win_count = 0
        tie_count = 0
        loss_count = 0
        for i in range(len(self.roundData)):
            if self.roundData[i][1] == 3:
                result = 'W'
                win_count += 1
            elif self.roundData[i][1] == 1:
                result = 'T'
                tie_count += 1
            else:
                result = 'L'
                loss_count += 1
                
        return win_count, loss_count, tie_count
        
    def printMatchHistory(self, player_list):
        print("<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>")
        print("Player " + str(self.id) + " (" + str(self.deck) + ") match history:")
        win_count = 0
        tie_count = 0
        loss_count = 0
        for i in range(len(self.roundData)):
            if self.roundData[i][1] == 3:
                result = 'W'
                win_count += 1
            elif self.roundData[i][1] == 1:
                result = 'T'
                tie_count += 1
            else:
                result = 'L'
                loss_count += 1
            print("Round " + str(i) + ": " + result + " vs " + player_list[self.roundData[i][0]].deck + \
                  " - " + str(win_count) + "/" + str(loss_count) + "/" + str(tie_count) + \
                  " (vs Player " + str(player_list[self.roundData[i][0]].id) + ")")
        print("Final score: " + str(self.score) + " (" + str(win_count) + "/" + str(loss_count) + "/" + str(tie_count) + ")")

In [5]:
### STANDINGS GENERATION (PRINTING ONLY) ###

def PrintStandings(standings):
    for player in standings:
        print("Player " + str(player[0]) + " - Score: " + str(player[1]))
        continue

def GenerateStandings(player_list,id_list_sorted):
    standings = {}
    for i in id_list_sorted:
        standings[i] = player_list[i].score
    
    standings_sorted = sorted(standings.items(), key=lambda item: item[1], reverse=True)
    
    #PrintStandings(standings_sorted)
    
    return standings_sorted

### GENERATE RESULT ###
# weighted random result for a given match of two decks
def GenerateResult(deck1, deck2, deck_objects,elim):
    winrate = deck_objects[deck1].winrates[deck2]
    tierate = deck_objects[deck1].tierates[deck2]
    
    if not elim:
        number = random.randint(0, 1000)
        if number <= winrate * 10:
            return 3, 0
        elif number <= (winrate + tierate) * 10:
            return 1, 1
        else:
            return 0, 3
    else:
        number = random.randint(0, 1000)
        if number <= (winrate + tierate/2) * 10:
            return 3, 0
        else:
            return 0, 3
    
### PROCESS PAIRINGS ###
# round independent
def ProcessPairings(pairings, player_list, deck_objects, elim):
    for pairing in pairings:
        player1 = player_list[pairing[0]]
        player2 = player_list[pairing[1]]

        deck1 = player1.deck
        deck2 = player2.deck

        result_p1, result_p2 = GenerateResult(deck1, deck2, deck_objects, elim)
        #print(player1.id, deck1, result_p1, player2.id, deck2, result_p2)

        player1.createRound(player2.id, result_p1)
        player2.createRound(player1.id, result_p2)
        
    return 1

def ProcessPairingsTopCut(pairings, player_list, deck_objects, id_list_sorted, elim):
    pairing_results = []
    for pairing in pairings:
        player1ID = id_list_sorted[pairing[0]-1] 
        player2ID = id_list_sorted[pairing[1]-1] 
        
        if player2ID == -1: #auto win
            pairing_results.append(pairing[0])
        
        else:
 
            player1 = player_list[player1ID]
            player2 = player_list[player2ID]

            deck1 = player1.deck
            deck2 = player2.deck

            result_p1, result_p2 = GenerateResult(deck1, deck2, deck_objects, elim)
            
            if result_p1 == 3:
                pairing_results.append(pairing[0])
            else:
                pairing_results.append(pairing[1])
        
    return pairing_results

### GET RESULTS FOR DECKS ###
def ProcessDecks(player_list, deck_objects, day2):
    deck_results = []
    for target_deck in deck_objects:
        score = 0
        number_of_players = 0
        for player in player_list:
            if player.deck == deck_objects[target_deck].name:
                if day2:
                    if player.score> 25:
                        #score += player.score
                        score += 1
                        number_of_players += 1
                else:
                    if player.score> 15:
                        #score += player.score
                        score += 1
                        number_of_players += 1
        #if:
            #average_score = round(score/number_of_players,1)
        #else:
            #average_score = 0
        average_score = score
        #print(deck_objects[target_deck].name + ': ' + str(average_score))
        deck_results.append([deck_objects[target_deck].name, average_score])
    return deck_results

### PRINT MATCH HISTORY FOR A SPECIFIC DECK ###
def PrintDeckMatchHistory(players_list, deck):
    total_decks_day1 = 0
    total_decks_day2 = 0
    for player in players_list:
        if player.deck == deck:
            player.printMatchHistory(players_list)
            total_decks_day1 += 1
            
            if len(player.roundData) > total_rounds:
                total_decks_day2 += 1
    print("Total results for " + str(deck) + ": " + str(total_decks_day1) + " players in Day 1, " + str(total_decks_day2) + " players in Day 2.")

In [6]:
### GENERATING A ROUND AFTER FIRST ###

def GeneratePairings(number_players, player_list, id_list, id_list_sorted):
    performance_matrix = np.ones([number_players, number_players], dtype=int)*(-10)

    # generating performance matrix
    # loop through the lines
    for i in id_list:
        # need to convert to actual player IDs
        id_i = id_list_sorted[i]

        # loop through the columns
        for j in range(i,len(id_list)):
            id_j = id_list_sorted[j]        

            # filling in performances for each pairing (added scores)
            performance = player_list[id_i].score + player_list[id_j].score

            # if paired against itself
            if id_i == id_j:
                performance = -10

            # if paired against a previous opponent
            if player_list[id_i].id in player_list[id_j].previousOpps:
                performance = -10

            performance_matrix[i][j] = performance

    #print(id_list_sorted)
    #print(performance_matrix)

    # [preemptive] list for possible pairing solutions
    solutions = []

    # list for pairings chosen
    pairings = []

    # looping through lines
    for i in range(0,len(id_list)-1):
    #for i in range(1):

        # first value that is not -10
        reference_value = np.max(performance_matrix[i])

        if reference_value == -10:
            continue

        # lists for possible pairings (and respective indices)
        valid_pairings = []
        valid_indices = []

        # looping through columns
        for j in range(i+1,len(id_list)):

            # a pairing is valid if it is equal to the first column value
            if performance_matrix[i][j] == reference_value:
                player1 = id_list_sorted[i]
                player2 = id_list_sorted[j]

                valid_pairings.append([player1, player2])
                valid_indices.append([i, j])

            # eliminating the first player from the pool
            performance_matrix[i][j] = -10

        #print(valid_pairings)

        # choosing one pairing at random from the available ones
        chosen_pairing = random.randint(0, len(valid_pairings)-1)
        pairings.append(valid_pairings[chosen_pairing])

        #print(pairings)

        # eliminating the second player from the pool
        for k in range(len(id_list)):
            column = valid_indices[chosen_pairing][1]
            line = valid_indices[chosen_pairing][1]
            performance_matrix[k][column] = -10
            performance_matrix[line][k] = -10

        #print(performance_matrix)

    if len(pairings) < 5:
        print("WARNING")
    return pairings

In [7]:
class Tournament():
    def __init__(self, metashare_df, deck_objects):
        self.metashare = metashare_df
        self.deck_objects = deck_objects
        
        self.deck_results = [] # for single tournament at end of swiss rounds
        
        self.current_round = 1
        self.players_list = []
        self.day2 = False
        self.top8 = 0
        self.winner = 0
        player_id = 0

        for i in range(len(self.metashare.index)):
            deck_number = self.metashare.at[i, 'players']
    
            for j in range(int(deck_number)):
                self.players_list.append(Player(player_id, self.metashare.at[i, 'deck']))
                player_id += 1
                
    def printRoundData(self, pairings):
        print("----- ROUND " + str(self.current_round) + " -----")
        #print("Pairings: " + str(pairings))

        print("\nResults:")
        ProcessPairings(pairings, self.players_list, self.deck_objects, False)
        self.deck_results = ProcessDecks(self.players_list, self.deck_objects, self.day2)

        print("\nStandings:")
        standings = GenerateStandings(self.players_list,self.id_list_sorted)
        self.id_list_sorted = [tp[0] for tp in standings]
        self.current_round += 1
                
    def runRound1(self, id_list, number_tables):
        round1_list = copy.deepcopy(id_list)
        self.id_list_sorted = copy.deepcopy(id_list)
        pairings = []
        for i in range(number_tables):
            player1 = round1_list[0]
            round1_list.pop(0)

            p2_index = random.randint(0,len(round1_list)-1)
            player2 = round1_list[p2_index]
            round1_list.pop(p2_index)

            pairings.append([player1, player2])
        
        self.printRoundData(pairings)

    def runRounds(self, id_list, total_rounds):
        while self.current_round <= total_rounds:
            pairings = GeneratePairings(number_players, self.players_list, id_list, self.id_list_sorted)
            self.printRoundData(pairings)
            
    def selectDay2(self):
        print("\n----- DAY 2 -----")
        id_list_sorted_day2 = []
        for i in self.id_list_sorted:
            if self.players_list[i].score > 15:
                id_list_sorted_day2.append(i)
        id_list_day2 = np.arange(0,len(id_list_sorted_day2),1).tolist()
        self.id_list_sorted = id_list_sorted_day2
        self.current_round = 1
        self.day2 = True
        
        return id_list_day2
    
    def topCut(self):
        print("\n----- ASYMMETRICAL TOP CUT -----")
        #----- calculate Opps winrate (only needed for top cut) ------#
        standings = []
        same_scores = []
        #score of 1st seed to use as first reference for sorting same scores
        score_ref = self.players_list[self.id_list_sorted[0]].score
        #go through all players in day 2
        for i in self.id_list_sorted:
            #start new line if the score is different than what is in reference
            if self.players_list[i].score != score_ref:
                standings.append(same_scores)
                same_scores = []
                score_ref = self.players_list[i].score #update new reference score
            
            #initialize win counts 
            wins = 0
            totals = 0
            #go through all the opponents and count their wins
            for opp in self.players_list[i].previousOpps:
                results = self.players_list[opp].matchHistory() #instead of this function, we can implement saving this info
                wins += results[0]
                totals += results[0] + results[1] + results[2]
            #calculate winrate
            OppsWin = wins/totals*100
            #save this value for sorting later
            same_scores.append([i,OppsWin])
            #We could start a variable in Player to store this info, but for now it is not implemented
            #We have also not implemented OppsOpps winrate
        
        #reorder same scores
        new_standings = []
        for score in standings:
            score_sorted = sorted(score, key=lambda item: item[1], reverse=True)
            new_standings += score_sorted
        #new standings ordered by Opps winrate for each score level
        self.id_list_sorted = [tp[0] for tp in new_standings]
            
        #----- Who is in asymmetric top cut? -----#
        AsyPlayers = []
        #add top 7 already
        for i in range(0,7):
            player_id = self.id_list_sorted[i]
            AsyPlayers.append(self.players_list[player_id].id)
        
        i = 7
        player_id = self.id_list_sorted[i] #8th seed id
        cutOff = self.players_list[player_id].score #save score of 8th seed
        
        #go through all players with the same score
        while self.players_list[self.id_list_sorted[i]].score == cutOff:
            player_id = self.id_list_sorted[i]
            i += 1
            AsyPlayers.append(self.players_list[player_id].id) #add this to the asymmetric list
        
        #remove extra seats above top 32
        if len(AsyPlayers)>32:
            diff = len(AsyPlayers) - 32
            AsyPlayers = AsyPlayers[:-diff]
        
        #fill empty seats until top 32
        i=len(AsyPlayers)
        while i< 32:
            i+= 1
            AsyPlayers.append(-1) #consider -1 as empty
        
        #----- Determine Top 32 ------#
        print("  ----- Top 32 determination -----")
        top32 = [[1,32], [16,17], [8,25], [9,24], [4,29], [13,20], [5,28], [12,21], [2,31], [15,18], [7,26], [10,23], [3,30], [14,19], [6,27], [11,22]]
        i = 1
        top = top32
        #go over the 5 rounds for top 32
        while i <= 5:
            #print("     ----- Top pairings: " + str(top))
            if i==3: #save top8 decks
                print("        ----- Top 8: ")
                top_decks = []
                for j in range(0,4):
                    player_id0 = AsyPlayers[top[j][0]-1]
                    player_id1 = AsyPlayers[top[j][1]-1]
                    deck0 = self.players_list[player_id0].deck
                    deck1 = self.players_list[player_id1].deck
                    top_decks.append(deck0)
                    top_decks.append(deck1)
                    print(deck0 + " vs " + deck1)
                self.top8 = top_decks 
                
            if i==4:
                print("        ----- Top 4: ")
                for j in range(0,2):
                    player_id0 = AsyPlayers[top[j][0]-1]
                    player_id1 = AsyPlayers[top[j][1]-1]
                    deck0 = self.players_list[player_id0].deck
                    deck1 = self.players_list[player_id1].deck
                    print(deck0 + " vs " + deck1)
                
            if i==5:
                print("        ----- Finals: ")
                for j in range(0,1):
                    player_id0 = AsyPlayers[top[j][0]-1]
                    player_id1 = AsyPlayers[top[j][1]-1]
                    deck0 = self.players_list[player_id0].deck
                    deck1 = self.players_list[player_id1].deck
                    print(deck0 + " vs " + deck1)
            
            #determine results with elim=True (ties cannot happen)
            winners = ProcessPairingsTopCut(top, self.players_list, self.deck_objects, AsyPlayers, True)
            top = []  
            #find new pairings
            if i<5: #do not find new pairings if last round
                for j in range(0,len(winners)-1,2):
                    top.append([winners[j],winners[j+1]])
            i += 1
        
        winner_deck = self.players_list[AsyPlayers[winners[0]-1]].deck
        self.winner = winner_deck
        print("  ----- Winner: Player " + str(AsyPlayers[winners[0]-1]) + " playing " + winner_deck)

In [8]:
### DEFINE METASHARE ###
metashare_df = pd.read_csv(r'C:\Users\sofia\My Drive\Code\Python\PokemonTCG\Tournament Simulation\Metashare.csv')

metashare_df['players'] = np.floor(metashare_df['percentage']/100 * number_players)

#print(metashare_df)

# adjust number of Zards
zard_location = metashare_df.index[metashare_df['deck'] == 'charizard-pidgeot'].tolist()[0]
extra_zards = number_players - metashare_df['players'].sum()

metashare_df.at[zard_location, 'players'] = metashare_df.at[zard_location, 'players'] + extra_zards

print(metashare_df)
print(metashare_df['players'].sum())

                     deck  percentage  players
0             ancient-box    0.611621     12.0
1           archaludon-ex    7.951070    159.0
2       banette-gardevoir    0.611621     12.0
3            ceruledge-ex    0.611621     12.0
4       charizard-pidgeot   12.293578    254.0
5      dragapult-dusknoir    0.611621     12.0
6            dragapult-ex    0.611621     12.0
7   dragapult-iron-thorns    7.706422    154.0
8       dragapult-pidgeot    0.611621     12.0
9         gardevoir-ex-sv    7.951070    159.0
10           gholdengo-ex    7.217125    144.0
11         iron-thorns-ex    0.611621     12.0
12        klawf-terapagos    4.220183     84.0
13          lost-zone-box    0.611621     12.0
14         lugia-archeops    4.892966     97.0
15            miraidon-ex    4.525994     90.0
16        palkia-dusknoir    0.611621     12.0
17        pidgeot-control    0.611621     12.0
18    raging-bolt-ogerpon    8.868502    177.0
19        regidrago-vstar   16.146789    322.0
20        roa

In [9]:
### IMPORT WINRATES ###
winrates_df = pd.read_csv(r'C:\Users\sofia\My Drive\Code\Python\PokemonTCG\Tournament Simulation\Winrates.csv').set_index('Opponent')
winrates_df = winrates_df.fillna(50.0)

#print(winrates_df)

tierates_df = pd.read_csv(r'C:\Users\sofia\My Drive\Code\Python\PokemonTCG\Tournament Simulation\Tierates.csv').set_index('Opponent')
tierates_df = tierates_df.fillna(0.0)

#print(tierates_df)

### CREATING DECKS AND STORING MATCHUP DATA ###
deck_names = winrates_df.index
deck_objects = {}
for deck in deck_names:
    deck_objects[deck] = Deck(deck)
    
for deck in deck_objects:
    deck_objects[deck].generateMatchupData(winrates_df, tierates_df)


In [15]:
### RUNNING TOURNAMENT ###
all_results = []
all_results_day2 = []
all_results_top8 = []
all_results_winner = []
nr = 100

for t in range(nr):
    print("\n----- TOURNAMENT " + str(t) + " -----")
    tournament = Tournament(metashare_df, deck_objects)
    tournament.runRound1(id_list, number_tables)
    tournament.runRounds(id_list, total_rounds)
    all_results.append(tournament.deck_results)
    id_list_day2 = tournament.selectDay2()
    tournament.runRounds(id_list_day2, 4)
    all_results_day2.append(tournament.deck_results)
    tournament.topCut()
    all_results_top8.append(tournament.top8)
    all_results_winner.append(tournament.winner)


----- TOURNAMENT 0 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
regidrago-vstar vs gholdengo-ex
iron-thorns-ex vs charizard-pidgeot
regidrago-vstar vs iron-thorns-ex
archaludon-ex vs archaludon-ex
        ----- Top 4: 
regidrago-vstar vs iron-thorns-ex
iron-thorns-ex vs archaludon-ex
        ----- Finals: 
regidrago-vstar vs iron-thorns-ex
  ----- Winner: Player 948 playing iron-thorns-ex

----- TOURNAMENT 1 -----
----- 

----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
snorlax-stall vs gardevoir-ex-sv
regidrago-vstar vs lugia-archeops
regidrago-vstar vs dragapult-iron-thorns
gardevoir-ex-sv vs regidrago-vstar
        ----- Top 4: 
gardevoir-ex-sv vs lugia-archeops
dragapult-iron-thorns vs gardevoir-ex-sv
        ----- Finals: 
gardevoir-ex-sv vs dragapult-iron-thorns
  ----- Winner: Player 735 playing gardevoir-ex-sv

----- TOURNAMENT 9 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
-----

----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
ancient-box vs terapagos-dusknoir
roaring-moon-ex vs gardevoir-ex-sv
charizard-pidgeot vs dragapult-iron-thorns
charizard-pidgeot vs regidrago-vstar
        ----- Top 4: 
terapagos-dusknoir vs gardevoir-ex-sv
charizard-pidgeot vs charizard-pidgeot
        ----- Finals: 
gardevoir-ex-sv vs charizard-pidgeot
  ----- Winner: Player 196 playing charizard-pidgeot

----- TOURNAMENT 18 -----
----- ROUND 1 -----

Results:

Standings


Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
snorlax-stall vs archaludon-ex
roaring-moon-ex vs regidrago-vstar
gardevoir-ex-sv vs charizard-pidgeot
klawf-terapagos vs charizard-pidgeot
        ----- Top 4: 
snorlax-stall vs regidrago-vstar
gardevoir-ex-sv vs klawf-terapagos
        ----- Finals: 
regidrago-vstar vs klawf-terapagos
  ----- Winner: Player 1607 playing regidrago-vstar

----- TOURNAMENT 26 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standing

----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
gardevoir-ex-sv vs roaring-moon-ex
regidrago-vstar vs dragapult-iron-thorns
lost-zone-box vs raging-bolt-ogerpon
regidrago-vstar vs raging-bolt-ogerpon
        ----- Top 4: 
roaring-moon-ex vs dragapult-iron-thorns
lost-zone-box vs regidrago-vstar
        ----- Finals: 
dragapult-iron-thorns vs regidrago-vstar
  ----- Winner: Player 549 playing dragapult-iron-thorns

----- TOURNAMENT 35 -----
----- ROUND 1 -----

Results:

S

...

----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
iron-thorns-ex vs gardevoir-ex-sv
gholdengo-ex vs lugia-archeops
archaludon-ex vs charizard-pidgeot
archaludon-ex vs gholdengo-ex
        ----- Top 4: 
iron-thorns-ex vs lugia-archeops
archaludon-ex vs gholdengo-ex
        ----- Finals: 
iron-thorns-ex vs gholdengo-ex
  ----- Winner: Player 949 playing iron-thorns-ex

----- TOURNAMENT 60 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings

----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
archaludon-ex vs raging-bolt-ogerpon
gardevoir-ex-sv vs gholdengo-ex
charizard-pidgeot vs regidrago-vstar
dragapult-iron-thorns vs lugia-archeops
        ----- Top 4: 
raging-bolt-ogerpon vs gholdengo-ex
regidrago-vstar vs lugia-archeops
        ----- Finals: 
gholdengo-ex vs regidrago-vstar
  ----- Winner: Player 892 playing gholdengo-ex

----- TOURNAMENT 69 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 ----

----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
gholdengo-ex vs regidrago-vstar
lost-zone-box vs regidrago-vstar
gholdengo-ex vs regidrago-vstar
klawf-terapagos vs lost-zone-box
        ----- Top 4: 
regidrago-vstar vs regidrago-vstar
regidrago-vstar vs klawf-terapagos
        ----- Finals: 
regidrago-vstar vs klawf-terapagos
  ----- Winner: Player 1690 playing regidrago-vstar

----- TOURNAMENT 77 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Result

----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
pidgeot-control vs gardevoir-ex-sv
klawf-terapagos vs palkia-dusknoir
raging-bolt-ogerpon vs gardevoir-ex-sv
archaludon-ex vs terapagos-dusknoir
        ----- Top 4: 
gardevoir-ex-sv vs klawf-terapagos
raging-bolt-ogerpon vs archaludon-ex
        ----- Finals: 
gardevoir-ex-sv vs raging-bolt-ogerpon
  ----- Winner: Player 701 playing gardevoir-ex-sv

----- TOURNAMENT 86 -----
----- ROUND 1 -----

Results:

Standings:
----- R

Standings:
----- ROUND 4 -----

Results:

Standings:

----- ASYMMETRICAL TOP CUT -----
  ----- Top 32 determination -----
        ----- Top 8: 
gardevoir-ex-sv vs gardevoir-ex-sv
lost-zone-box vs lugia-archeops
archaludon-ex vs gardevoir-ex-sv
gardevoir-ex-sv vs charizard-pidgeot
        ----- Top 4: 
gardevoir-ex-sv vs lugia-archeops
archaludon-ex vs gardevoir-ex-sv
        ----- Finals: 
gardevoir-ex-sv vs gardevoir-ex-sv
  ----- Winner: Player 686 playing gardevoir-ex-sv

----- TOURNAMENT 94 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4 -----

Results:

Standings:
----- ROUND 5 -----

Results:

Standings:
----- ROUND 6 -----

Results:

Standings:
----- ROUND 7 -----

Results:

Standings:
----- ROUND 8 -----

Results:

Standings:

----- DAY 2 -----
----- ROUND 1 -----

Results:

Standings:
----- ROUND 2 -----

Results:

Standings:
----- ROUND 3 -----

Results:

Standings:
----- ROUND 4

In [20]:
final_statistics = []
statistics = np.zeros(len(all_results[0]))

for i in range(len(all_results)):
    for j in range(len(all_results[i])):
        statistics[j] += all_results[i][j][1]
        
for i in range(0,len(statistics)):
    final_statistics.append([all_results[0][i][0],statistics[i]/nr])
    #final_statistics.append([all_results[0][i][0],statistics[i]/nr/metashare_df['players'][i]*100])


final_statistics.sort(key=lambda x: x[1])
print(final_statistics)

with open('SimulationResultsDay2.csv', 'w', newline='', encoding='utf-8') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in final_statistics:
        writer.writerow([key, float(value)])

[['dragapult-ex', 0.44], ['banette-gardevoir', 0.65], ['ceruledge-ex', 0.85], ['dragapult-dusknoir', 1.21], ['dragapult-pidgeot', 1.22], ['pidgeot-control', 1.51], ['ancient-box', 1.64], ['palkia-dusknoir', 1.87], ['iron-thorns-ex', 2.09], ['lost-zone-box', 2.22], ['snorlax-stall', 3.15], ['miraidon-ex', 6.45], ['lugia-archeops', 12.39], ['terapagos-dusknoir', 13.9], ['roaring-moon-ex', 15.31], ['klawf-terapagos', 17.25], ['raging-bolt-ogerpon', 23.74], ['dragapult-iron-thorns', 29.61], ['gholdengo-ex', 30.6], ['archaludon-ex', 32.97], ['gardevoir-ex-sv', 35.48], ['charizard-pidgeot', 40.82], ['regidrago-vstar', 55.95]]


In [21]:
final_statistics = []
statistics = np.zeros(len(all_results_day2[0]))

for i in range(len(all_results_day2)):
    for j in range(len(all_results_day2[i])):
        statistics[j] += all_results_day2[i][j][1]
        
for i in range(0,len(statistics)):
    final_statistics.append([all_results_day2[0][i][0],statistics[i]/nr])
    #final_statistics.append([all_results_day2[0][i][0],statistics[i]/nr/metashare_df['players'][i]*100])


final_statistics.sort(key=lambda x: x[1])
print(final_statistics)

with open('SimulationResultsTop64.csv', 'w', newline='', encoding='utf-8') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in final_statistics:
        writer.writerow([key, float(value)])

[['dragapult-ex', 0.01], ['ceruledge-ex', 0.12], ['banette-gardevoir', 0.14], ['dragapult-dusknoir', 0.22], ['dragapult-pidgeot', 0.29], ['ancient-box', 0.42], ['pidgeot-control', 0.45], ['palkia-dusknoir', 0.46], ['iron-thorns-ex', 0.66], ['lost-zone-box', 0.67], ['miraidon-ex', 1.01], ['snorlax-stall', 1.37], ['lugia-archeops', 3.26], ['roaring-moon-ex', 3.38], ['terapagos-dusknoir', 3.61], ['klawf-terapagos', 5.78], ['raging-bolt-ogerpon', 6.01], ['dragapult-iron-thorns', 8.61], ['gholdengo-ex', 9.67], ['archaludon-ex', 10.18], ['charizard-pidgeot', 11.28], ['gardevoir-ex-sv', 11.74], ['regidrago-vstar', 16.45]]


In [22]:
#flatten top 8
all_results_top8_new = [x for xs in all_results_top8 for x in xs]

keys = list(Counter(all_results_top8_new).keys())
values = list(Counter(all_results_top8_new).values())
print(keys)
print(values) 

with open('SimulationResultsTop8.csv', 'w', newline='', encoding='utf-8') as csv_file:  
    writer = csv.writer(csv_file)
    for i in range(0,len(keys)):
        ind = metashare_df.loc[metashare_df['deck'] == keys[i],'players']
        #writer.writerow([keys[i], float(values[i])/nr/float(ind)*100])
        writer.writerow([keys[i], float(values[i])/nr])

['regidrago-vstar', 'gholdengo-ex', 'iron-thorns-ex', 'charizard-pidgeot', 'archaludon-ex', 'palkia-dusknoir', 'ancient-box', 'pidgeot-control', 'raging-bolt-ogerpon', 'dragapult-iron-thorns', 'klawf-terapagos', 'terapagos-dusknoir', 'lugia-archeops', 'roaring-moon-ex', 'snorlax-stall', 'gardevoir-ex-sv', 'miraidon-ex', 'lost-zone-box', 'dragapult-pidgeot']
[136, 89, 10, 82, 79, 7, 7, 7, 39, 60, 46, 22, 28, 33, 18, 125, 3, 8, 1]


In [23]:
keys = list(Counter(all_results_winner).keys())
values = list(Counter(all_results_winner).values())
print(keys)
print(values) 

with open('SimulationResultsWinner.csv', 'w', newline='', encoding='utf-8') as csv_file:  
    writer = csv.writer(csv_file)
    for i in range(0,len(keys)):
        ind = metashare_df.loc[metashare_df['deck'] == keys[i],'players']
        #writer.writerow([keys[i], float(values[i])/nr/float(ind)*100])
        writer.writerow([keys[i], float(values[i])/nr])

['iron-thorns-ex', 'gholdengo-ex', 'regidrago-vstar', 'archaludon-ex', 'raging-bolt-ogerpon', 'klawf-terapagos', 'gardevoir-ex-sv', 'roaring-moon-ex', 'snorlax-stall', 'charizard-pidgeot', 'dragapult-iron-thorns', 'lost-zone-box', 'dragapult-pidgeot', 'lugia-archeops', 'terapagos-dusknoir', 'pidgeot-control']
[3, 6, 20, 11, 6, 7, 20, 3, 2, 10, 5, 1, 1, 2, 1, 2]


In [22]:
# print match history for a given player
tournament.players_list[141].printMatchHistory(tournament.players_list)
tournament.players_list[193].printMatchHistory(tournament.players_list)
tournament.players_list[580].printMatchHistory(tournament.players_list)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Player 141 (archaludon-ex) match history:
Round 0: W vs charizard-pidgeot - 1/0/0 (vs Player 264)
Round 1: L vs terapagos-dusknoir - 1/1/0 (vs Player 1393)
Round 2: W vs regidrago-vstar - 2/1/0 (vs Player 1160)
Round 3: W vs regidrago-vstar - 3/1/0 (vs Player 1202)
Round 4: L vs ceruledge-ex - 3/2/0 (vs Player 184)
Round 5: W vs archaludon-ex - 4/2/0 (vs Player 88)
Round 6: W vs lost-zone-box - 5/2/0 (vs Player 826)
Round 7: W vs regidrago-vstar - 6/2/0 (vs Player 1146)
Round 8: W vs roaring-moon-ex - 7/2/0 (vs Player 1297)
Round 9: W vs gardevoir-ex-sv - 8/2/0 (vs Player 500)
Round 10: W vs dragapult-dusknoir - 9/2/0 (vs Player 363)
Round 11: W vs lost-zone-box - 10/2/0 (vs Player 769)
Final score: 30 (10/2/0)
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Player 193 (ceruledge-ex) match history:
Round 0: L vs klawf-terapagos - 0/1/0 (vs Player 681)
Round 1: W vs regidrago-vstar - 1/1/0 (vs Player 1129)

In [23]:
# print match history for all players with a given deck
PrintDeckMatchHistory(tournament.players_list, 'archaludon-ex')

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Player 37 (archaludon-ex) match history:
Round 0: W vs dragapult-iron-thorns - 1/0/0 (vs Player 404)
Round 1: L vs miraidon-ex - 1/1/0 (vs Player 986)
Round 2: L vs charizard-pidgeot - 1/2/0 (vs Player 322)
Round 3: W vs snorlax-stall - 2/2/0 (vs Player 1336)
Round 4: L vs snorlax-stall - 2/3/0 (vs Player 1328)
Round 5: T vs lost-zone-box - 2/3/1 (vs Player 798)
Round 6: W vs miraidon-ex - 3/3/1 (vs Player 960)
Round 7: L vs regidrago-vstar - 3/4/1 (vs Player 1234)
Final score: 10 (3/4/1)
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Player 38 (archaludon-ex) match history:
Round 0: L vs miraidon-ex - 0/1/0 (vs Player 956)
Round 1: L vs dragapult-iron-thorns - 0/2/0 (vs Player 435)
Round 2: L vs archaludon-ex - 0/3/0 (vs Player 145)
Round 3: W vs archaludon-ex - 1/3/0 (vs Player 112)
Round 4: L vs regidrago-vstar - 1/4/0 (vs Player 1270)
Round 5: L vs lost-zone-box - 1/5/0 (vs Player 833)
Round 6: W vs 